# CNN Image - cats and dogs (from kaggle)

In [ ]:
import os, shutil
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import Image, display

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers

from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.preprocessing import image

In [ ]:
# select the folders with the images:

# class folder
train_cats_dir = 'data/training_set/cats'
train_dogs_dir = 'data/training_set/dogs'

validation_cats_dir = 'data/validation_dir/cats'
validation_dogs_dir = 'data/validation_dir/dogs'

test_cats_dir = 'data/test_set/cats'
test_dogs_dir = 'data/test_set/dogs'

# main folders
train_dir = 'data/training_set/'
validation_dir = 'data/validation_dir'
test_dir = 'data/test_set'

In [ ]:
# Sanity Check

print ('total training cat images:', len(os.listdir(train_cats_dir)))

print('total training dog images:', len(os.listdir(train_dogs_dir)))

print ('total validation cat images:', len(os.listdir(validation_cats_dir)))

print ('total validation dog images:', len(os.listdir(validation_dogs_dir)))

print('total test cat images:', len(os.listdir(test_cats_dir)))

print('total test dog images:', len(os.listdir(test_dogs_dir)))

## Data preprocessing
1) Read the picture files. <br>
2) Decode the JPEG content to RGB grids of pixels. <br>
3) Convert these into floating point tensors. <br>
4) Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know, neural networks prefer to deal with small input values). <br> <br>
Keras has a module for image processing

### Without Data Augmentation

In [ ]:
# Rescale all images by 1/255
# Train
train_datagen = ImageDataGenerator(rescale=1./255)
# Validation
test_datagen = ImageDataGenerator(rescale=1./255)

# this generator will continuously create from this directory
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150,150), #size of the final image
                                                   batch_size=20, #will return batches of 20 images at each time
                                                   class_mode='binary') #binary classification (if it was multiclass = categorical)

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                       target_size=(150,150),
                                                       batch_size=20,
                                                       class_mode='binary',
                                                       shuffle=False)

In [ ]:
# Sanity Check
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

### With Data Augmentation

In [ ]:
# do the data augmentation only for the training data

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# this generator will continuously create from this directory
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(64,64), #size of the final image
                                                   batch_size=1, #will return batches of 20 images at each time
                                                   class_mode='binary') #binary classification (if it was multiclass = categorical)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(64,64),
                                                  batch_size=1,
                                                  class_mode='binary',
                                                  shuffle=False)

In [ ]:
# Sanity Check
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

## Building the network

In [ ]:
model = models.Sequential()

# Conv 1
# size of the image 150x150 (3 RGB), 32 neurons, (3,3) strides 
model.add(layers.Conv2D(32,
                         (3,3),
                         activation='relu',
                         input_shape=(150, 150, 3)))
# MaxPooling 1
model.add(layers.MaxPooling2D((2,2)))
# Conv 2
model.add(layers.Conv2D(64, (3,3), activation='relu'))
# MaxPooling 2
model.add(layers.MaxPooling2D((2,2)))
# Conv 3
model.add(layers.Conv2D(128, (3,3), activation='relu'))
#  MaxPooling 3
model.add(layers.MaxPooling2D((2,2)))
# Conv 4
model.add(layers.Conv2D(128, (3,3), activation='relu'))
#  MaxPooling 4
model.add(layers.MaxPooling2D((2,2)))
# Fully connected NN
model.add(layers.Flatten())
# Dense Layer 1
model.add(layers.Dense(512, activation='relu'))
# Output layer - sigmoid for binary and softmax for multiclass
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
# Architecture of the NN
model.summary()

In [ ]:
# Defining Loss | Optimizer | Metrics
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc']) # for accuracy

## Training the model

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=100, #depends on the size of the batch and te total images for training (steps_per_epoch * batch_size = size of training set)
                    epochs=30,
                    validation_data=validation_generator,
                    validation_steps=50)  #depends on the size of the batch and te total images for validation (validation_steps * batch_size = size of validation set)

# save the model
model.save('cats_and_dogs_small_1.h5')

## Plot the results

In [ ]:
# Names of the metrics saved on history dictionary

history_dict = history.history
history_dict.keys()

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Heatmap - class activation heatmap for an image classification model

In [ ]:
# The Grad-CAM algorithm

def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
# function to overlap the heatmap and the real image

def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))

In [ ]:
# Test in one image prodiction - define the parameters:

# name form model.summary() of the last CNN layer
last_conv_layer_name = 'conv2d_1'

# The local path to our target image
img_path = 'data/test_set/dogs/dog.4010.jpg'

# Size of the training
img_size = (64,64)
display(Image(img_path))

In [ ]:
# Display the image

# Prepare image
img_array = get_img_array(img_path, size=img_size)

# Make model
# model = model_builder(weights="imagenet")

# Remove last layer's softmax
model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
print("Predicted:", np.argmax(preds[0]))

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

save_and_display_gradcam(img_path, heatmap)